In [17]:
#imports
import os, sys, json, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
import xlsxwriter
from tqdm import tqdm
from datetime import date #date/time manipulation
import glob as glob

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None  # default='warn'
from IPython.display import display_markdown


from cprl_functions.state_capture import thi_states,state_ref, state_coding, state_coding_r, state_pat, state_abv_pat
from cprl_functions.text_printing import bordered
from cprl_functions.defined_functions import create_pk, add_seats, get_key, get_recent_file


In [18]:

#going to fpk_assigned folder
os.chdir(r"C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\full_pk_assigned")
files = glob.glob('*')

print(files)


['IL_com_info.xlsx', 'in_com_info.xlsx', 'KS_com_info.xlsx', 'MO_com_info.xlsx', 'NC_coms.xlsx', 'nd_com_info.xlsx', 'oh_com_info.xlsx', 'ok_com_info.xlsx', 'va_com_info.xlsx']


In [26]:

dfs = []
for file in files:
    df = pd.read_excel(file)
    print(file)
    # print(df.head())
    for i,j in enumerate(df.columns):
        # print(j)
        if 'com' in str(j):
            com_name_col_index = i
            com_name_col_name = j
        elif re.search(f'full_pk|fpk',str(j).strip()):
            fpk_col_index = i
            fpk_col_name = j
        elif str(j).strip() == 'role' or str(j).strip() == 'position':
            role_col_index = i
            role_col_name = j

    
    state_match = re.match(r'^([A-Za-z]{2})_com',str(file))
    # print(type(state_match.group(1).upper()))
    state = state_match.group(1).upper()
    
    # re.match()

    print(f'full_pk column: {fpk_col_name}')
    print(f'com name column: {com_name_col_name}')
    print(f'role column: {role_col_name}')
    print(df.columns)
    
    
    ext_df = df.loc[:,[fpk_col_name, com_name_col_name, role_col_name]]
    ext_df = ext_df.rename(columns={fpk_col_name: 'fpk', com_name_col_name: 'com_name', role_col_name: 'role'})
    ext_df['state'] = state
    # 42 1 123 01
    ext_df['extracted'] = ext_df['fpk'].astype(str).str.extract(r'^\d{2}(\d{1})\d{5}')
    
    print()
    print(list(set(ext_df['extracted'].to_list())))
    mapping = {"01":"Senate", "00":"House"}
    ext_df['chamber'] = ext_df['extracted'].map(mapping).fillna('')




    dfs.append(ext_df)
    print(f"{'\n'}=============================")

    # full_pk
    # committee Name
    # position
    


IL_com_info.xlsx
full_pk column: full_pk
com name column: com name
role column: role
Index(['full_pk', 'chamber', 'com name', 'name', 'role', 'district'], dtype='object')

['1', '0']

in_com_info.xlsx
full_pk column: full_pk
com name column: com
role column: position
Index(['full_pk', 'chamber', 'com', 'name', 'district', 'position'], dtype='object')

['1', '0']

KS_com_info.xlsx
full_pk column: full_pk
com name column: com name
role column: role
Index(['full_pk', 'chamber', 'com name', 'name', 'role', 'district'], dtype='object')

['1']

MO_com_info.xlsx
full_pk column: fpk
com name column: com_name
role column: role
Index(['fpk', 'chamber', 'com_name', 'name', 'role', 'district'], dtype='object')

['1', '0']

NC_coms.xlsx
full_pk column: full_pk
com name column: committee
role column: position
Index(['full_pk', 'chamber', 'committee', 'name', 'position'], dtype='object')

['1', '0']

nd_com_info.xlsx
full_pk column: fpk
com name column: com_name
role column: position
Index(['fpk', 'c

In [27]:

com_file = pd.concat(dfs).reset_index(drop = True)
com_file = com_file.drop(['extracted', 'chamber'], axis = 1)


In [29]:

# com_file.to_excel(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\aggregated\comms_info.xlsx', index = False)
# file_name_cb = f'compiled_plus_bills{str(date.today()).replace('-','_')}.xlsx'

com_file.to_excel(r'C:\Users\clutz\THE HUNT INSTITUTE\The Hunt Institute Team Site - Documents\Development (formerly Grants Management)\!Administrative\Christian\THII\legislator data\committee info\committee update files\aggregated\comms_info.xlsx',sheet_name=f'com_info_{str(date.today()).replace('-','_')}', index=False)